In [1]:
import numpy as np

In [144]:
H_1 = np.loadtxt('H1.txt', dtype=int)
y_1 = np.loadtxt('y1.txt', dtype=int)

### Question 1
Write a function that receives a parity check matrix $H$ and builds a systematic encoding matrix $G$ for it. This may require altering $H$ by swapping columns, so the function should return two matrices: $\hat{H}$ and $G$, such that $\hat{H}$ is equal to $H$ up to a column permutation and $HGt = 0$ for all $t$ (all the operations are performed in $F2$). Print the outputs of the function for the following matrix:

In [56]:
H = np.array([
    [1, 1, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 1],
    [1, 0, 0, 1, 1, 0]
], dtype=int)

First, create a function that generates $\hat{H}$ given $H$; this is achieved through Gaussian elimination.

In [127]:
def get_H_hat(H):
    N, K = H.shape[1], H.shape[1] - H.shape[0]
    H_hat = np.copy(H)
    for i in range(0, N - K):
        j = 1
        while H_hat[i, K + i] != 1:
            H_hat[i] = np.mod(H_hat[i] + H_hat[np.mod(i + j, N - K)], 2)
            j += 1
        for j in range(0, N - K):
            if H_hat[j, K + i] == 1 and j != i:
                H_hat[j] = np.mod(H_hat[j] + H_hat[i], 2)
    return H_hat

Then create the required function that given the parity-check matrix $H$ it returns the generator matrix $G$ (along with $\hat{H}$).

In [128]:
def get_identity_matrix(n):
    I = np.zeros((n, n), dtype=int)
    for i in range(n):
        I[i, i] = 1
    return I

def encode(H):
    N, K = H.shape[1], H.shape[1] - H.shape[0]
    H_hat = get_H_hat(H)
    P = np.transpose(H_hat[:,:K])
    I = get_identity_matrix(K)
    G = np.concatenate((I, P), axis=1)
    return H_hat, G

In [151]:
H_hat, G = encode(H_1)

### Question 3
Write an LDPC-decoder based on Loopy Belief Propagation for Binary Symmetric Channel. Specifically, write a function that receives a parity check matrix $\hat{H}$, a received word $y$, a noise ratio $p$ and an optional parameter of a maximum number of iterations (with default value of $20$). The function should return a decoded vector along with the following return code: $0$ for success, $−1$ if the maximum number of iterations is reached without a successful decoding. Try to make your code efficient. Print the result of the decoding for a given parity check matrix $H_1$ (in H1.txt) and vector $y_1$ (in y1.txt). The noise ratio was $p = 0.1$. How many iterations did your algorithm take to converge?

Let us create a function that checks the validity of a message against a parity-check matrix.

In [161]:
def is_valid(m, H):
    return np.sum(np.mod(H @ m, 2)) == 0

In [162]:
def p_y_given_x(x, y, p):
    """ Returns the probability of the original message being x given the observed x. """
    probability = 1
    for n in range(0, len(x)):
        probability *= (p ** (x[n] - y[n])) * ((1 - p) ** (x[n] - y[n] + 1))
        

def decode(H_hat, y, p, maximum_iterations=20):
    pass